# Obtención de datos mediante API

La API [Neo-Browse](https://api.nasa.gov/) es un servicio web proporcionado por la NASA dentro de su conjunto de APIs de NeoWs (Near Earth Object Web Service). Está diseñada para ofrecer datos sobre asteroides cercanos a la Tierra (NEOs, por sus siglas en inglés) en un formato accesible para desarrolladores, investigadores y entusiastas de la astronomía. Esta API permite navegar por un catálogo completo de asteroides, obteniendo información detallada sobre cada uno de ellos.

Características principales de la API Neo-Browse

1.	Navegación por el catálogo de asteroides:
	- Permite explorar todos los objetos cercanos a la Tierra conocidos en el catálogo de la NASA. A diferencia de otras APIs de NeoWs que se enfocan en rangos de fechas específicos, Neo-Browse no requiere parámetros de tiempo.
2.	Paginación:
	- Los datos están organizados en páginas para facilitar la navegación por el catálogo. Puedes controlar el número de resultados por página y cambiar entre páginas con los parámetros size y page.
3.	Información detallada sobre los asteroides:
	- La API proporciona datos importantes como:
	    - Nombre del asteroide.
	    - Diámetro estimado (mínimo y máximo) en diferentes unidades (metros, kilómetros, etc.).
        - Velocidad relativa.
	    - Distancia a la Tierra.
        - Probabilidad de ser “potencialmente peligroso” para la Tierra.
	    - Datos orbitales (distancia más cercana al Sol, inclinación orbital, etc.).
4.	Identificación única de asteroides:
    - Cada asteroide tiene un ID único que puede ser usado para buscar información más detallada en otras APIs de NeoWs.


La librería `request` es fundamental para realizar solicitudes HTTP. Es ligera, fácil de usar y ampliamente utilizada para consumir APIs.

In [1]:
import requests 
import pandas as pd

https://api.nasa.gov/neo/rest/v1/neo/browse es el endpoint para la NeoWs (Near Earth Object Web Service). Cada API de la NASA tiene un endpoint diferente; consulta la documentación oficial para más detalles.

In [2]:
endpoint = "https://api.nasa.gov/neo/rest/v1/neo/browse"

Usamos `requests.get()` para enviar la solicitud HTTP. Le pasamos la URL base y los parámetros como argumentos.

Comprobamos si el código de estado HTTP (`status_code`) es 200, lo que indica que la solicitud fue exitosa.

In [3]:
r = requests.get(endpoint)
r.status_code

403

En el intento anterior el código es 403, ya que no se agregó el API_KEY, el cual es obligatorio por el API de la NASA. Podemos complementar el resultado obtenido al imprimir el contendio con el método `json()`.

In [4]:
r.json()

{'error': {'code': 'API_KEY_MISSING',
  'message': 'No api_key was supplied. Get one at https://api.nasa.gov:443'}}

Intentaremos nuevamente una conexión, pero ahora prepararemos el API_KEY.

La clave es necesaria para autenticar las solicitudes. Usamos API_KEY en el ejemplo, pero debes registrarte para obtener tu propia clave si planeas hacer muchas solicitudes.

Para esto, los parámetros se envían como un diccionario (params en este caso). Incluyen la clave API y otros filtros como la fecha para la imagen astronómica, etc.

In [5]:
payload = {'api_key' : '3riqw90MyFDjJ3nEvSUGAIYrCn8bZrtYJ50xFeck'}
endpoint = "https://api.nasa.gov/neo/rest/v1/neo/browse"

r = requests.get(endpoint, params=payload)
r.status_code


200

En este segundo intento, el resultado de `status_code` es igual a 200, lo que significa que fue exitosa la solicitud de GET.

Como el resultado fue exitoso, convertimos el contenido a un objeto JSON con .json() y accedemos a los campos que queremos (por ejemplo, links, page y near_earth_objetcts).

In [ ]:
j = r.json()

In [ ]:
print(j)

A continuación, exploraremos más a detalle el resultado de `json()`. Mediante el método `keys()` obtenemos una lista de las claves del objeto JSON.

In [7]:
r.json().keys()

dict_keys(['links', 'page', 'near_earth_objects'])

Al imprimir el valor de la clave `link` observamos que el contenido es otro diccionario, el cual indica el link de la siguiente página a recuperar (`next`) y el link del recurso actual (`self`). 

In [8]:
j['links']

{'next': 'http://api.nasa.gov/neo/rest/v1/neo/browse?page=1&size=20&api_key=3riqw90MyFDjJ3nEvSUGAIYrCn8bZrtYJ50xFeck',
 'self': 'http://api.nasa.gov/neo/rest/v1/neo/browse?page=0&size=20&api_key=3riqw90MyFDjJ3nEvSUGAIYrCn8bZrtYJ50xFeck'}

En el caso del valor a sociado a la clave `page`, podemos diponer del tamaño de la página recuperada. En este caso contien 20 registros (`size`). En adición, podemos conocer el número total de element (`total_elements`), las páginas totales (`total_pages`) y el número de pagina actual (`number`).

In [9]:
j['page']

{'size': 20, 'total_elements': 37872, 'total_pages': 1894, 'number': 0}

En la tercera clave `near_earth_objects` encontraresmos los datos de interes. Nuevamen observamos que la información vienen contenida como un diccionario anidado. 

In [ ]:
j['near_earth_objects']

Ahora que conocemos el la estructura de los datos recuperados en formato JSON, realizaremos una conversión para la construcción de un DataFrame en pandas. 

Primero, utilizaremos el método `json_normalize()` de panda para convertir datos en formato JSON (particularmente aquellos con estructuras jerárquicas o anidadas) en un DataFrame de Pandas, que es una estructura tabular de filas y columnas.

En partícular solo nos interesa lo contenido en la clave `'near_earth_objects'`. Una vez realizada la conversión con el método `pd.from_dict()` creamos el DataFrame a apartir del diccionario.


In [10]:
normalized = pd.json_normalize(j['near_earth_objects'])
df = pd.DataFrame.from_dict(normalized)
df.head()

,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_range,orbital_data.orbit_class.orbit_class_description
0,2000433,2000433,433 Eros (A898 PA),Eros,433,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,10.41,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,...,1.133458052992745,178.9102909699136,1.782904615277399,2460445.664780181111,86.66755145250509,.5597405522715084,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...
1,2000719,2000719,719 Albert (A911 TB),Albert,719,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,15.59,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,...,1.19476035548719,156.2155261769503,4.07755393959166,2459955.833573002386,148.4506773209914,.2302751796962134,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,13.84,False,"[{'close_approach_date': '1974-01-04', 'close_...",False,...,1.060482619836633,350.4742461301673,3.884567382350691,2460678.609991922119,340.1984298231293,.2535087981652097,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,9.20,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,...,1.245158481030607,132.4961444468926,4.08543903828178,2460569.662511270184,6.984964961069339,.2265088776284111,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...
4,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.38,False,"[{'close_approach_date': '1908-03-14', 'close_...",False,...,1.085428152631312,26.7176005107723,2.754879623203948,2460838.872819526800,271.7008031502767,.3704247700094682,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...


Una vez que hemos logrado construir un DataFrame a apartir de una petición GET a la API de la NASA. Ahora relizaremos una solicitude diez páginas. 

In [11]:
for _ in range(0, 10):
    r = requests.get(endpoint, params=payload)
    if r.status_code == 200:
        json = r.json()

        data = json['near_earth_objects']
        new_link = json['links']['next']

        endpoint = new_link

En el código anterior se realizan diez peticiones GET seguidas a la API de la NASA, lo que podría ocasionar problemas por la forma inmediata y recurrente. Una buena práctiva al realizar peticiones repetitivas, es incluir un `time.sleep()` para realizar una pausa con la finalidad de evitar las peticiones casí simultaneas a la API.

Para esto utilizaremos la función `sleep()` del módulo `time` para definir una espera de cinco segundos.

In [12]:
import time

In [13]:
for _ in range(0, 10):
    r = requests.get(endpoint, params=payload)
    if r.status_code == 200:
        json = r.json()

        data = json['near_earth_objects']
        new_link = json['links']['next']

        endpoint = new_link
    
    time.sleep(5)

El código, obtenido en el paso anterior, ahora será modificada para almacenar las páginas solicitadas en un diccionario `dict_datos`, con la finalidad de almacenar todas las peticiones realizadas a la API.

In [14]:
dict_datos = {}

for i in range(0, 10):
    r = requests.get(endpoint, params=payload)
    if r.status_code == 200:
        json = r.json()

        data = json['near_earth_objects']
        dict_datos[i] = data

        new_link = json['links']['next']
        endpoint = new_link
    
    time.sleep(5)

In [ ]:
dict_datos

Para mejorar el código anterior, es recomendable utilizar `try-except` con la finalida de capturar cualquier excepción cue ocurriera durante la petición. En este momento, solo vamos a capturar la excepción y continuar (en futuras implementaciones, se recomienda mejorar el código, para intentar nuevamente recuperar la página y no saltarla).

In [15]:
dict_datos = {}

for i in range(0, 10):
    try:
        time.sleep(5)
        r = requests.get(endpoint, params=payload)
        if r.status_code == 200:
            json = r.json()
            data = json['near_earth_objects']
            dict_datos[i] = data
            new_link = json['links']['next']
            endpoint = new_link

    except:
        continue    
    
    

En este punto, tenemos un diccionario `dict_datos` con los resultados de las peticiones realizadas en el bloque de código anterior. Ahora es necesario convertir cada uno de los resultados en un `DataFrame`.

In [16]:
for k in dict_datos:
    normalized = pd.json_normalize(dict_datos[k])
    df = pd.DataFrame.from_dict(normalized)
    dict_datos[k] = df

A continuación, se ilustra el resultado obtenido al imprimir el primer dataframe contenido en `dict_datos[0]`.

In [17]:
first_df = dict_datos[0]
first_df.head(5)

,id,neo_reference_id,name,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,...,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_range,orbital_data.orbit_class.orbit_class_description
0,2153951,2153951,153951 (2002 AC3),153951,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.75,False,"[{'close_approach_date': '1900-01-27', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153951?ap...,...,1.208573069580133,215.1607014571422,2.464018747771989,2460549.375423559195,20.24975611319781,.3960865306462564,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...
1,2153953,2153953,153953 (2002 AD9),153953,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,16.84,False,"[{'close_approach_date': '1912-03-02', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153953?ap...,...,.3389143676536511,9.802678801425939,3.208007888394581,2460833.808552051953,262.6351449422293,.4173222721646719,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...
2,2153957,2153957,153957 (2002 AB29),153957,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.76,False,"[{'close_approach_date': '1937-09-20', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153957?ap...,...,.6103133076617502,73.16235036980498,4.45311881444916,2461168.222396435118,221.09508722705,.2446704826957179,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...
3,2153958,2153958,153958 (2002 AM31),153958,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.36,True,"[{'close_approach_date': '1903-07-19', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153958?ap...,...,.9340356772558281,197.9578148586224,2.472051027854558,2460224.166298512855,166.8931957227103,.4434712997087535,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...
4,2154007,2154007,154007 (2002 BY),154007,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.12,False,"[{'close_approach_date': '1904-04-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2154007?ap...,...,1.186625053993438,23.76615777115104,2.449287374243072,2460528.296699024118,29.03260453976844,.4020952525351443,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...


A continuación vamos a convetir el valor de cada uno de los elementos del diccionario `dict_datos` en un elemento de una lista.

In [18]:
lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

In [ ]:
lista_de_dataframes

El método `concat` de la biblioteca Pandas se utiliza para unir o concatenar objetos de tipo DataFrame o Series a lo largo de un eje (filas o columnas). Es muy útil cuando necesitas combinar datos de múltiples fuentes o dividir y luego recombinar tus datos para análisis.

Principales usos del método concat

1.	Concatenar a lo largo de las filas (axis=0): Esto apila los objetos uno debajo del otro, agregando nuevas filas al DataFrame o Series resultante.
2.	Concatenar a lo largo de las columnas (axis=1): Esto combina objetos lado a lado, agregando nuevas columnas`

A continuación, realizaremos la concatenación de cada uno de los elementos de la lista `lista_de_dataframes` como un solo DataFrame.

In [19]:
df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)

In [20]:
df_completo.head(5)

,id,neo_reference_id,name,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,...,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_range,orbital_data.orbit_class.orbit_class_description,name_limited
0,2153951,2153951,153951 (2002 AC3),153951,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.75,False,"[{'close_approach_date': '1900-01-27', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153951?ap...,...,215.1607014571422,2.464018747771989,2460549.375423559195,20.24975611319781,.3960865306462564,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...,NaN
1,2153953,2153953,153953 (2002 AD9),153953,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,16.84,False,"[{'close_approach_date': '1912-03-02', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153953?ap...,...,9.802678801425939,3.208007888394581,2460833.808552051953,262.6351449422293,.4173222721646719,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...,NaN
2,2153957,2153957,153957 (2002 AB29),153957,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,17.76,False,"[{'close_approach_date': '1937-09-20', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153957?ap...,...,73.16235036980498,4.45311881444916,2461168.222396435118,221.09508722705,.2446704826957179,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...,NaN
3,2153958,2153958,153958 (2002 AM31),153958,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.36,True,"[{'close_approach_date': '1903-07-19', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2153958?ap...,...,197.9578148586224,2.472051027854558,2460224.166298512855,166.8931957227103,.4434712997087535,J2000,APO,a (semi-major axis) > 1.0 AU; q (perihelion) <...,Near-Earth asteroid orbits which cross the Ear...,NaN
4,2154007,2154007,154007 (2002 BY),154007,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,18.12,False,"[{'close_approach_date': '1904-04-06', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2154007?ap...,...,23.76615777115104,2.449287374243072,2460528.296699024118,29.03260453976844,.4020952525351443,J2000,AMO,1.017 AU < q (perihelion) < 1.3 AU,Near-Earth asteroid orbits similar to that of ...,NaN


Ahora que hemos unificado todos nuestro datos en un solo DataFrame, lo guardaremos con el formato `csv` para realizar otras actividades relacionadas con la compresión de los datos y la preparación de los datos.

In [21]:
df_completo.to_csv('../datasets/near_earth_objects-raw.csv')